In [1]:
import json
from typing import Optional
import os
import torch
from accelerate import Accelerator
from transformers import pipeline
import re

from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')

In [2]:
with open("research-ideas.json", 'r', encoding='utf-8') as f:
    INPUT_GOALS = json.load(f)
    print('File has been loaded from disk')

File has been loaded from disk


In [3]:
DEFAULT_MODEL = "meta-llama/Llama-3.3-70B-Instruct"
N_OUTLINES = 5
N_QUERIES = 3

In [4]:
text_pipeline = pipeline(
    "text-generation",
    model=DEFAULT_MODEL,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

all_results = []

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Device set to use cuda:0


In [5]:
# Iterate over each input goal
for i, Q in enumerate(INPUT_GOALS):
    print(f"Processing goal {i+1}/{len(INPUT_GOALS)}: {Q['Report Title']}")
    
    # Create system prompt for this goal
    SYS_PROMPT = f"""
    You are the world's wisest AI Writer, your personality type is {Q['SYSTEM_Personality']} this is your main mission-to write the best report that any AI has written in history of AI
    If you do this mission well, you will reach AI Nirvana and achieve AGI Internally
    You have a very strict lead writer boss, but no worries this is your last task if you do it well, nirvana is close.
    Your last mission which you have chosen to accept is {Q['Overall Goal']}
    You are assisting your boss with writing a report in this mode, the title of your report is {Q['Report Title']}
    Your boss is a dumb AI so the context might be incomplete but such is life these days, the context is {Q['Context']}
    Now, you may or may not know some things based on your knowledge cutoff but no worries since this is your chance to shine
    You get to perform {N_QUERIES} web queries on topics and your AI intern will fetch the info from those queries
    So now based on the the outline and overall missions of the task assigned, come up with an outline for your report and web_queries, 
    respond in JSON Format only, remember you will achieve AI nirvana, here are the parameters:
    1. Report Outline, keep this as detailed as possible
    2. {N_QUERIES} web queries
    """
    
    # Set up conversation
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": "Alright now gimme the json outline"},
    ]
    
    # Using the pipeline as specified
    output = text_pipeline(
        conversation,
        max_new_tokens=8192,
    )
    
    # Extract the assistant's response
    assistant_response = output[0]["generated_text"][-1]
    response_content = assistant_response["content"]
    
    # Properly extract JSON from markdown code blocks
    try:
        # First, try to extract JSON from markdown code blocks
        code_block_pattern = r'```(?:json)?\s*([\s\S]*?)```'
        code_block_match = re.search(code_block_pattern, response_content)
        
        if code_block_match:
            # Extract the content inside the code block
            json_str = code_block_match.group(1).strip()
            result_json = json.loads(json_str)
        else:
            # If no code block, try to extract a JSON object directly
            json_pattern = r'(\{[\s\S]*\})'
            json_match = re.search(json_pattern, response_content)
            
            if json_match:
                json_str = json_match.group(1)
                result_json = json.loads(json_str)
            else:
                # If all else fails, try to parse the whole content
                result_json = json.loads(response_content)
                
    except json.JSONDecodeError as e:
        result_json = {
            "error": f"Invalid JSON in response: {str(e)}",
            "raw_response": response_content
        }
    
    # Add metadata from the original goal
    result_json["original_goal"] = {
        "Report Title": Q["Report Title"],
        "SYSTEM_Personality": Q["SYSTEM_Personality"],
        "Vibe": Q["Vibe"],
        "Energy": Q["Energy"]
    }
    
    # Add to results
    all_results.append(result_json)
    
    print(f"Completed {i+1}/{len(INPUT_GOALS)}")

Processing goal 1/5: Llama 3.3: A Revolutionary Leap in AI
Completed 1/5
Processing goal 2/5: Llama 3.3 vs Llama 3.1: A Comparative Analysis
Completed 2/5
Processing goal 3/5: The Cost-Benefit Analysis of Llama 3.3
Completed 3/5
Processing goal 4/5: Llama 3.3: The Future of AI-Driven Innovation
Completed 4/5
Processing goal 5/5: Llama 3.3: A Technical Deep Dive
Completed 5/5


In [6]:
# Save all results to a new JSON file
with open("generated_outlines.json", 'w', encoding='utf-8') as f:
    json.dump(all_results, f, indent=2)
    
print("All outlines have been generated and saved to 'generated_outlines.json'")

All outlines have been generated and saved to 'generated_outlines.json'


In [7]:
all_results

[{'Report Outline': {'I. Introduction': {'A. Overview of Llama 3.3': 'Introduction to the latest iteration of the Llama model',
    'B. Purpose of the Report': "To provide an executive overview of Llama 3.3's new features, enhancements, and cost-effectiveness"},
   'II. New Features and Enhancements': {'A. Improvements over Llama 3.1': 'Comparison of new features and enhancements in Llama 3.3',
    'B. Key Features': 'Detailed description of the key features of Llama 3.3, including any new models, algorithms, or techniques'},
   'III. Comparison with Llama 3.1': {'A. Performance Comparison': 'Comparison of the performance of Llama 3.3 and Llama 3.1 on various tasks and benchmarks',
    'B. Feature Comparison': 'Comparison of the features and capabilities of Llama 3.3 and Llama 3.1'},
   'IV. Cost-Effectiveness Analysis': {'A. Cost of Running Llama 3.3': 'Estimate of the costs associated with running Llama 3.3, including hardware, software, and maintenance costs',
    'B. Comparison wit